In [ ]:
from cyipopt import minimize_ipopt
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import Bounds, LinearConstraint, NonlinearConstraint

In [ ]:
# Fahrzeug
m = 790 #kg
F_MotorMax = 10.41 #kN
alpha = 0.001872 # 1/m

def F_HaftMax(age:str="new"):
    if age=="old":
        return 9.88 #kN
    elif age=="new":
        return 12.27 

# Config Constants
L = 3
B = 1
dL = 1
xStart = 0.1
vStart = 0.2
N=int(L/dL)
num_vars_t =2
optimization_start_t = 1

u = np.zeros((L,num_vars_t)) # 0=x, 1=v
u[0] = [xStart, vStart]


In [ ]:
# Functions
def s(u: np.ndarray, t:int):
    """strecke"""
    x = u[:,0]
    return np.sqrt(dL**2 + (x[t-1]-x[t])**2)

def vm(u:np.ndarray, t:int):
    """durchschnittsgeschwindigkeit"""
    v = u[:,1]
    return (v[t-1] + v[t]) / 2

def dt(u:np.ndarray, t:int):
    """Gebrauchte Zeit für den abschnitt"""
    return s(u, t) / vm(u, t)

def a(u:np.ndarray, t:int):
    """Beschleunigung """
    return (u[t,1]- u[t-1,1]) / dt(u,t)    

In [ ]:
# Objective function
# zeit minimieren
def objective(u:np.ndarray):
    return np.sum([dt(u, t) for t in range(1, L)])

In [ ]:
# Non linear Constraints
def F_Brems_Motor(u:np.ndarray, t:int):
    u = u.reshape((2,N))
    return m * (a(u, t)+ alpha* vm(u, t)**2)

def F_Reifen(u:np.ndarray, t:int):
    return m* a(u, t)

In [ ]:
constraints = [{'type': 'ineq', 'fun': F_Brems_Motor, 'args': (t,)} for t in range(1, L)]
constraints += [{"type": "ineq", "fun": F_Reifen, "args": (t,)} for t in range(1, L)]

In [ ]:
# Bounds are made on input varaibles (x,v) in this case

# 0 <= x(i) <= B
# 0.001 <= v(i) <= infty
# bounds for each decision variable. has shape (n,), None for infinity
# since the variables need to be in a vector of shape (n,) we need to concatenate them

bounds = Bounds(
    lb=[0 for _ in range(optimization_start_t, N)] + [0.001 for _ in range(optimization_start_t, N)],
    ub=[B for _ in range(optimization_start_t, N)] + [np.inf for _ in range(optimization_start_t, N)]
)


In [ ]:

result = minimize_ipopt(
    fun=objective, 
    x0=u[0], 
    bounds=box_constraints, 
    constraints=constraints
)